In [ ]:
%run Projekt.ipynb

In [ ]:
dataP.info()

In [ ]:
dataO.info()

In [ ]:
len(dataO.name.unique())

In [ ]:
namesO = dataO["name"].unique().sort()

In [ ]:
namesP = dataP["name"].unique().sort()

In [ ]:
np.array_equal(namesO, namesP)

In [ ]:
dataO.head()

In [ ]:
    dataOtypes = dataO["kurtosis_oxygen"].str.isdecimal()

In [ ]:
dataOtypes.shape

In [ ]:
dataOtypes.head()

In [ ]:
duplNames = dataO[dataO['name'].duplicated(keep=False)]
duplNames.sort_values('name')

Po vypísani duplicitných mien sme zistili, že zlúčením týchto duplikátov vieme vyplniť chýbajúce hodnoty. Zároveň hodnoty, ktoré majú oba duplikáty sú rovnaké. 

In [ ]:
def fusionDuplicates(datasetO):
    duplicat = datasetO[datasetO['name'].duplicated(keep=False)]
    duplicat = duplicat.sort_values('name')
    duplicat = duplicat.reset_index(drop=True)
    
    for i, row in duplicat.iterrows():
        if(i%2==1):
            duplicat = duplicat.drop(i)
            continue
        for j, value in row.items():
            dupValue = duplicat.loc[i+1,j]
            if(pd.isna(value)):
                duplicat.loc[i,j]=dupValue
    datasetO.drop_duplicates(subset='name',keep=False,inplace=True)
    result = datasetO.append(duplicat, ignore_index=True)
    return(result)

In [ ]:
dataO=fusionDuplicates(dataO)
dataO

In [ ]:
dataO.info()

In [ ]:
dataO[dataO.isnull().any(axis=1)]

In [ ]:
replaced = dataO.replace(to_replace=r'^[0-9,\-].*[0-9]$',value='number',regex=True)
replaced.select_dtypes(include='object')

In [ ]:
replaced['kurtosis_oxygen'].unique()

In [ ]:
replaced['occupation'].unique()

In [ ]:
replaced['pregnant'].unique()

In [ ]:
replaced['education-num'].unique()

In [ ]:
replaced['edu'].unique()

In [ ]:
replaced['income'].unique()

In [ ]:
replaced['workclass'].unique()

In [ ]:
replaced['ethn'].unique()

In [ ]:
replaced['state'].unique()

In [ ]:
replaced['status'].unique()

In [ ]:
replaced['fam_memb'].unique()
replacet.

In [ ]:
def correctValuesO(dataset):
    for i, row in dataset.iterrows():
        for j, value in row.items():
            replValue = value.str.replace('_','-')
            dataset.loc[i,j]=replValue

In [ ]:
correctValuesO(dataO)